In [1]:
import datetime
import pandas as pd
import numpy as np
import argparse
import sys
import string
import re
import MeCab
import mojimoji

sys.path.append('../..')
from utils.func import norm_time

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--train_date', type=int, default=2016)
parser.add_argument('--test_date', type=int, default=2017)
# parser.add_argument('--code', type=list, default=['1711', '1712'])
# parser.add_argument('--code', type=list, default=['7203', '9984'])
# parser.add_argument('--code', type=list, 
#                     default=['9984'])

# TOPIX core30
parser.add_argument('--code', type=list, 
                    default=['2914', '3382', '4063', '4452', '4502', '4503',
                                    '4568', '6098', '6501',  '6758', '6861', '6954',
                                    '6981', '7203', '7267',  '7751', '7974', '8031',
                                    '8058', '8306', '8316',  '8411', '8766', '8802',
                                    '9020', '9022', '9432',  '9433', '9437', '9984'])
# parser.add_argument('--code', type=list, default=['8301', '7203', '9501', '6758', '9984',
#                                                  '8306', '8411', '6501', '6752', '6502'])

args = parser.parse_args(args=[])

code_list = [str(x) for x in args.code]

# テキストを取り出す

In [3]:
for i, date in enumerate(range(2013, 2020)):
    tmp = pd.read_csv('../../data/news/' + str(date) + '.csv', encoding='cp932')
    tmp = tmp[tmp['Company_IDs(TSE)'].isin(code_list)]
    # tmp = tmp[tmp['Company_Relevance'] == str(100)]
    tmp = tmp[['Time_Stamp_Original(JST)', 
                    'Company_Code(TSE)', 
                    'Headline', 
                   'Company_Name_J',
                    'News_Source',
                    'Company_Relevance', 
                    'Keyword_Article']]

    # 欠損除去
    tmp = tmp[~tmp["Keyword_Article"].isnull()]

    # タグ除去
    tmp = tmp[(tmp['News_Source'] == '日経') | 
                 (tmp['News_Source'] == 'ＮＱＮ') |
                 (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
                 (tmp['News_Source'] == 'Ｒ＆Ｉ')]

    tmp['code'] = tmp['Company_Code(TSE)'].astype(int)
    tmp['date'] = pd.to_datetime(tmp["Time_Stamp_Original(JST)"]).map(norm_time)
    tmp = tmp.set_index(['date', 'code'], drop=True)
    tmp = tmp.drop(['Time_Stamp_Original(JST)', 'News_Source', 'Company_Code(TSE)'], axis=1)

    if i == 0:
        df1 = tmp.copy()
    else:
        df1 = pd.concat([df1, tmp])


# 株価を取り出す

In [4]:
for i, code in enumerate(code_list):
    tmp = pd.read_csv('../../data/stock_price/' + str(code) + '.csv', index_col=0)
    tmp['code'] = int(code)
    if i == 0:
        df2 = tmp
    else:
        df2 = pd.concat([df2, tmp])

df2['date'] = pd.to_datetime(df2['date'])
df2 = df2.set_index(['date', 'code'], drop=True)
print(df2.shape)

(64320, 1)


# 銘柄名を取り出す

In [5]:
comp_json = {}
with open('../../data/company_name.csv') as f:
    line = f.readline()
    while line:
        split = line[:-1].split(',')
        comp_json[split[0]] = split[1:]
        line = f.readline()

# 形態素解析

In [6]:
m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd \
                                    -u ../../mydict/mydict.dic")

# /usr/local/libexec/mecab/mecab-dict-index \
# -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd \
# -u mydict.dic \
# -f utf-8 \
# -t utf-8 \
# mydict.csv

In [7]:
def Morph(row):
    comp_name = comp_json[str(row.name[1])]
    text = row['Headline']
    output_words = []

    # 前処理
    text = re.sub("(\*J)",  "", text)
    text = re.sub(" |　", "", text)
    text = mojimoji.zen_to_han(text, kana=False, digit=True)
    text = re.sub(r"\\u3000",  "", repr(text)[1:-1])
    text = re.sub(r"<.+>",  "", text)
    text = re.sub(r"＜.+＞",  "", text)
    text = re.sub(r"\(\d\)",  "", text)
    text = re.sub(r"|\.|（|）|「|」|｢|｣|―|：|【|】|~|\(|\)|。|/|:|\[|\]|,|\*|◇|☆|◎|;|&",  "", text)
    text = re.sub(r"\d+",  "0", text)

#     for p in string.punctuation:
#         text = text.replace(p, ' ')
    
    # company
    for name in comp_name:
        text = text.replace(name, '<company>')
    
    soup = m.parse(text)
    for i, row in enumerate(soup.split('\n')):
        category = row.split('\t')
        if category[0] == "EOS":
            break
        elif (i == 0) and (category[0]  =='人事'):
            break
        else:
            
            word = category[0]            
            tag = category[1].split(',')
            if tag[2] == '人名':
                word = '<person>'
            elif tag[2] == '地域':
                word = '<location>'
            elif tag[2] == '組織':
                if (category[0] != '東証') and (category[0] != '時価総額'):
                    word = '<organization>'
            elif (tag[0] == '助詞') or (word=='、') or \
                    (word=='､') or (word=='・') or (word=='･') or (word == ''):
                continue

            output_words.append(word)
    
    if len(output_words) < 4:
        return None
    else:
        return ':'.join(output_words)

In [8]:
arr1 = df1[0:200].apply(Morph, axis=1)
arr2 = df1[0:200]['Headline']

In [9]:
for x1,x2 in zip(arr1, arr2):
    print(x1)
    print(x2)
    print()

<organization>:米子会社:<location>:コンサル:会社:買収
<日経>◇日立の米子会社、英コンサル会社を買収

<company>:スマホ:また:通信:障害:データ:通信:でき:ず
<日経>◇ａｕスマホ、また通信障害　データ通信できず

<company>:全:グループ:海外:研修:将来:幹部:育成
<日経>◇三菱商事、全グループで海外研修　将来の幹部育成

<person>:<location>:知事:企業:立地:着実:自動車:0:税:撤廃:要請
<日経>◇大村・愛知知事「企業立地、着実に」　自動車２税撤廃要請

Uターンラッシュ:鉄道:空の便:とも:ピーク
<日経>◇Ｕターンラッシュ　鉄道・空の便ともにピーク

0年:<organization>:優秀:製品:サービス:賞:新た:な:着想:個性:光る
<日経>◇2012年日経優秀製品・サービス賞　新たな着想、個性光る

<company>:証券:社:<organization>:信:証:連携:中心:<company>:インベ:合併
<ＮＱＮ>◇みずほ証券社長「銀信証の連携の中心に」　みずほインベと合併

日経平均:大幅続伸:始まる:崖:回避:円安:好感:し:震災前:水準:上回る
<ＮＱＮ>◇日経平均、大幅続伸で始まる 「崖」回避と円安を好感し震災前水準上回る

日経平均:大幅続伸:始まる:崖:回避:円安:好感
<日経>◇日経平均、大幅続伸で始まる 「崖」回避と円安を好感

<location>:先物取引:委:<location>:<company>:証券:民事:制裁金:支払い:命令
<日経>◇米先物取引委、米国みずほ証券に民事制裁金支払い命令

<location>:<company>:証券:0万:円:罰金:<organization>:顧客:資産:管理:巡り
<ＮＱＮ>◇米国みずほ証券に1500万円罰金　ＣＦＴＣ、顧客資産の管理巡り

<company>:米:法人:機器:間:無線通信:合弁:設立
<日経>◇ソニー米法人、機器間無線通信で合弁設立

東証:0部:時価総額:0兆:円:上回る:0年0月:以来
<ＮＱＮ>◇東証１部の時価総額、300兆円上回る　11年３月以来

<company>:一時:0%:超:高:円安:進行:一段:採算:改善:期待
<ＮＱＮ>◇＜東証＞トヨタが一時５％超高　円安

In [10]:
re.sub(r"(\*J)", "", "ＩＰＯ：FUJIJPN(1449*J)　初値決定前の気配運用")

'ＩＰＯ：FUJIJPN(1449)\u3000初値決定前の気配運用'

In [11]:
"<QUICK>5%ﾙｰﾙ報告16日 日本ｱｸｱ(1429)―大量(ﾊﾝﾂﾏﾝ･ｲﾝﾍﾞｽﾄﾒﾝﾂ(ﾈｻﾞｰﾗﾝﾄﾞ)ﾋﾞｰｳﾞｨ)".replace("日本ｱｸｱ", "1")

'<QUICK>5%ﾙｰﾙ報告16日 1(1429)―大量(ﾊﾝﾂﾏﾝ･ｲﾝﾍﾞｽﾄﾒﾝﾂ(ﾈｻﾞｰﾗﾝﾄﾞ)ﾋﾞｰｳﾞｨ)'

In [12]:
df4 = df1.copy()
df4['Headline'] = df1.apply(Morph, axis=1)
df4 = df4.dropna(subset=['Headline'])

In [13]:
df4

Headline  \
date       code                                                     
2013-01-04 6501         <organization>:米子会社:<location>:コンサル:会社:買収   
           9433                <company>:スマホ:また:通信:障害:データ:通信:でき:ず   
           8058                   <company>:全:グループ:海外:研修:将来:幹部:育成   
2013-01-03 7203     <person>:<location>:知事:企業:立地:着実:自動車:0:税:撤廃:要請   
2013-01-05 9020                            Uターンラッシュ:鉄道:空の便:とも:ピーク   
...                                                           ...   
2019-05-19 8306               投信:基準:価額:0日:速報:値:更新:<company>:国際:投信   
           6758                 <company>:一時:0%:高:自社株:買い:好感:MS:提携   
           7203           <company>:<location>:スープラ:0年:ぶり:復活:0万:円   
           7267           <person>:<organization>:脱落:0月:上場:来:初:赤字   
           9022  気:なる:話題:0:生き:地獄:<company>:与野党:<organization>:新幹線   

                    Company_Name_J Company_Relevance  \
date       code                                        
2013-01-04 6501              日立製作所               100   
           9433               ＫＤＤＩ               100   
           8058               三菱商事               100   
2013-01-03 7203             トヨタ自動車                35   
2013-01-05 9020            東日本旅客鉄道                31   
...                            ...               ...   
2019-05-19 8306  三菱ＵＦＪフィナンシャル・グループ                50   
           6758                ソニー               100   
           7203             トヨタ自動車               100   
           7267             本田技研工業                28   
           9022             東海旅客鉄道                10   

                                                   Keyword_Article  
date       code                                                     
2013-01-04 6501  米国:子会社:ＣＥＯ:英国:コンサルティング会社:買収:海外:鉄道:情報収集:能力:提案:受...  
           9433  スマートフォン:スマホ:タブレット:データ通信:障害:発生:携帯電話:サービス:対応:端末:...  
           8058  投資:人材:育成:制度:１月:導入:本社:海外:合宿:次世代:リーダー:必要:知識:技術:英...  
2013-01-03 7203  愛知県:景気:対策:防災:減災:課題:活性化:自動車取得税:撤廃:国内生産:落ち込む:響く:...  
2013-01-05 9020  年末年始:鉄道:ピーク:駅舎:大きな:家族:大阪:往復:趣味:自転車:深く:名古屋市:岩手県...  
...                                                            ...  
2019-05-19 8306                                  ５月:運用:ファンド:サイト:速報  
           6758  大幅:反発:上昇:年初来高値:自社株買い:好感:買い:クラウド:ゲーム:終値:増加:株高:自...  
           7203  エンジン:価格:共同開発:新:モデル:投入:需要:喚起:代表:北米:登場:販売:環境意識:受...  
           7267  中国:インターネット:最大手:米国:スマートフォン:スマホ:対応:転落:シェア:下落:サービ...  
           9022  ブログ:金融市場:総務省:新幹線:事故:中国:人民銀:事情聴取:増税:民営化:値上げ:マスコ...  

[31523 rows x 4 columns]

In [14]:
m.parse('【牛さん熊さん】引け後：債券先物は買い先行後、閑散小動き・債券ディーリングルーム')

'【\t記号,括弧開,*,*,*,*,【,【,【\n牛さん\t名詞,一般,*,*,*,*,ウシサン,ウシサン,ウシサン\n熊さん\t名詞,固有名詞,一般,*,*,*,熊さん,クマサン,クマサン\n】\t記号,括弧閉,*,*,*,*,】,】,】\n引け\t名詞,一般,*,*,*,*,引け,ヒケ,ヒケ\n後\t名詞,接尾,副詞可能,*,*,*,後,ゴ,ゴ\n：\t記号,一般,*,*,*,*,：,：,：\n債券\t名詞,一般,*,*,*,*,債券,サイケン,サイケン\n先物\t名詞,一般,*,*,*,*,先物,サキモノ,サキモノ\nは\t助詞,係助詞,*,*,*,*,は,ハ,ワ\n買い先行\t名詞,固有名詞,一般,*,*,*,買い先行,カイセンコウ,カイセンコー\n後\t名詞,接尾,副詞可能,*,*,*,後,ゴ,ゴ\n、\t記号,読点,*,*,*,*,、,、,、\n閑散\t名詞,一般,*,*,*,*,閑散,カンサン,カンサン\n小動き\t名詞,固有名詞,一般,*,*,*,小動き,コウゴキ,コウゴキ\n・\t記号,一般,*,*,*,*,・,・,・\n債券\t名詞,一般,*,*,*,*,債券,サイケン,サイケン\nディーリング\t名詞,一般,*,*,*,*,ディーリング,ディーリング,ディーリング\nルーム\t名詞,一般,*,*,*,*,ルーム,ルーム,ルーム\nEOS\n'

# テキストと株価をくっつける

In [15]:
# 時系列をくっつける
df3 = pd.concat([df4, df2], axis=1, join_axes=[df4.index], levels=[0, 1])
df3 = df3.dropna()
df3 = df3.sort_values(by=['code', 'date'])
df3['Headline'] = \
    df3.groupby(level=[0,1]).apply(lambda x: ':<span>:'.join(list(x['Headline'])))

df3 = df3.dropna()
df3 = df3[~df3.duplicated(subset=['Headline'])]
df3['price'] = \
        df3['adj_close'].groupby(level=['code']).pct_change(1).shift(-1)*100
df3 = df3.dropna()

/home/ts-zemi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  


In [16]:
print(df3['price'].shape)
print(df3['price'][np.abs(df3['price']) > 1].shape)

(11725,)
(7380,)


In [17]:
# CSVファイルに保存する
df5 = pd.concat([df3[['Headline', 'price']].rename(
                                   columns={'Headline': 'state', 'price': 'reward'}),
                             df3[['Headline']].shift(-1).rename(
                                   columns={'Headline': 'next_state'})], axis=1).dropna()
df5 = df5[['state', 'next_state', 'reward']]

date_year = df5.index.map(lambda x: x[0].year)

df5[date_year <= args.train_date].to_csv(
    '../../data/news/text_train.tsv',
    header=None,
    index=None,
    sep='\t')

df5[(args.train_date < date_year) & (date_year < args.test_date)].to_csv(
    '../../data/news/text_val.tsv',
    header=None,
    index=None,
    sep='\t')

df5[(args.test_date <= date_year)].to_csv(
    '../../data/news/text_test.tsv',
    header=None,
    index=None,
    sep='\t')

# df5[(args.test_date <= date_year) & (date_year <= 2017)].to_csv(
#     '../../data/news/text_test.tsv',
#     header=None,
#     index=None,
#     sep='\t')

# 銘柄の名前リストを入れる

In [18]:
# for i, date in enumerate(range(2011, 2020)):
#     tmp = pd.read_csv('../../data/news/' + str(date) + '.csv', encoding='cp932')
# #     tmp = tmp[tmp['Company_IDs(TSE)'].isin(code_list)]
#     # tmp = tmp[tmp['Company_Relevance'] == str(100)]
#     tmp = tmp[['Time_Stamp_Original(JST)', 
#                     'Company_Code(TSE)', 
#                     'Headline', 
#                    'Company_Name_J',
#                     'News_Source',
#                     'Company_Relevance', 
#                     'Keyword_Article']]

#     # 欠損除去
#     tmp = tmp[~tmp["Keyword_Article"].isnull()]

#     # タグ除去
#     tmp = tmp[(tmp['News_Source'] == '日経') | 
#                  (tmp['News_Source'] == 'ＮＱＮ') |
#                  (tmp['News_Source'] == 'ＱＵＩＣＫ') | 
#                  (tmp['News_Source'] == 'Ｒ＆Ｉ')]

# #     tmp['code'] = tmp['Company_Code(TSE)'].astype(int)
# #     tmp = tmp.set_index(['date', 'code'], drop=True)
#     tmp = tmp.drop(['Time_Stamp_Original(JST)', 'News_Source'], axis=1)

#     if i == 0:
#         df5 = tmp.copy()
#     else:
#         df5 = pd.concat([df5, tmp])

In [19]:
# df5 = df5[['Company_Code(TSE)', 'Company_Name_J']].dropna()
# df5['Company_Code(TSE)'] = df5['Company_Code(TSE)'].astype(int)
# df5 = df5.sort_values('Company_Code(TSE)')

In [20]:
# df5[~df5.duplicated()].to_csv('../../data/company_name.csv', index=False, header=False)